### Web Scraping de Dados de Imovéis da Cidade de Rio das Ostras

#### 1 - Importando as Bibliotecas Necessárias

In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### 2 - Acesso e Controle do Navegador

In [2]:
URL_IMOVEIS = "https://www.vivareal.com.br/venda/rj/rio-das-ostras/" # -> URL a ser pesquisada

In [3]:
# Criar Instância do Google Chrome pelo Selenium
driver = webdriver.Chrome()
driver.implicitly_wait(10)

# Acesso ao Site através do método get
driver.get(URL_IMOVEIS)

#### 3 -  Acesso as classes HTML com os Dados

In [4]:
# Pegar Lista de Resultados dos Imoveis
resultados_imoveis = driver.find_elements_by_class_name('property-card__content')

<ipython-input-4-18d15999fbf3>:2: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  resultados_imoveis = driver.find_elements_by_class_name('property-card__content')


In [5]:
len(resultados_imoveis)

36

#### Preparando para Acessar as Janelas

In [6]:
# Setup wait for later
wait = WebDriverWait(driver, 10)

# Store the ID of the original window
original_window = driver.current_window_handle

#### 4 -  Loop para Acesso dos Sites (Cliques em Cada Link da Primeira Página)

In [7]:
metros_quadrados = []
quartos = []
banheiros = []
vagas = []

In [8]:
while True:
    # For para coleta de descrições
    for res in resultados_imoveis:
        
        assert len(driver.window_handles) == 1
        
        try:
        
            res.click() # Clicar na Descrição

            sleep(3)
            
            wait.until(EC.number_of_windows_to_be(2))
            
        except:
            
            pass
        
        for window_handle in driver.window_handles:
            
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                
                # Obtendo as variáveis selecionadas no site de acordo com a classe
            
                metros_v = driver.find_element_by_class_name('features__item--area')
                quartos_v = driver.find_element_by_class_name('features__item--bedroom')
                banheiros_v = driver.find_element_by_class_name('features__item--bathroom')
                vagas_v = driver.find_element_by_class_name('features__item--parking')
                
                # Quardando as info nas Listas
                
                metros_quadrados.append(metros_v.text)
                quartos.append(quartos_v.text)
                banheiros.append(banheiros_v.text)
                vagas.append(vagas_v.text)
                
                driver.close()
                
                driver.switch_to.window(original_window)
                
                break
                
    break         

<ipython-input-8-1ff00c0db9fd>:26: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  metros_v = driver.find_element_by_class_name('features__item--area')
<ipython-input-8-1ff00c0db9fd>:27: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  quartos_v = driver.find_element_by_class_name('features__item--bedroom')
<ipython-input-8-1ff00c0db9fd>:28: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  banheiros_v = driver.find_element_by_class_name('features__item--bathroom')
<ipython-input-8-1ff00c0db9fd>:29: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  vagas_v = driver.find_element_by_class_name('features__item--parking')


#### 5 - Tratando os Dados e Colocando em Um Data Frame

In [16]:
#len(metros_quadrados)
#len(quartos)
#len(banheiros) 
len(vagas) # Todas as listas possuem 35 itens coletados 

35

In [17]:
# Inserindo os dados em um dicionário
dados = {'Metragem': metros_quadrados , 'Quartos': quartos, 'Banheiros': banheiros, 'Vagas': vagas}

In [18]:
# Tranformando em Data Frame Pandas
dados_imoveis = pd.DataFrame(dados)

In [20]:
# Visualizando dados dos imoveis
dados_imoveis

,Metragem,Quartos,Banheiros,Vagas
0,64m²,2 quartos,2 banheiros\n1 suíte,1 vaga
1,66m²,2 quartos,1 banheiro,Não informado SOLICITAR
2,71m²,2 quartos,1 banheiro,1 vaga
3,180m²,3 quartos,3 banheiros\n1 suíte,2 vagas
4,58m²,2 quartos,2 banheiros\n1 suíte,1 vaga
5,90m²,2 quartos,3 banheiros\n2 suítes,1 vaga
6,66m²,2 quartos,1 banheiro,2 vagas
7,63m²,2 quartos,2 banheiros\n1 suíte,1 vaga
8,220m²,4 quartos,2 banheiros\n2 suítes,4 vagas
9,90m²,2 quartos,2 banheiros\n1 suíte,2 vagas


In [21]:
# Salvando Arquivo em CSV
dados_imoveis.to_csv('Dados_Imoveis_Rio_das_Ostras.csv')

#### FIM